<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex14notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex14notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo14.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の FFT の関数
from scipy.fft import fft, ifft

# 科学技術計算のライブラリ SciPy の中の WAVE ファイルを扱うモジュール
from scipy.io.wavfile import read, write

---
## 標本化と高速フーリエ変換
---



#### 準備

あとの実験で使う関数とデータを用意しておきます．

In [ ]:
# WAVE ファイルを読み込む関数
#
def readWAVE(filename):
    
    framerate, data = read(filename)

    # チャンネル数とフレーム数（データ点の数）を求める
    if data.ndim == 1:
        nchannels = 1
    else:
        nchannels = data.shape[1]
    nframes = data.shape[0]
    
    print('filename = ', filename)
    print('nchannels = ', nchannels)       # チャンネル数
    print('framerate = ', framerate)       # 標本化周波数
    print('nframes = ', nframes)             # フレーム数
    print('duration = {0:.2f}[sec]'.format(nframes / framerate))   # 長さ（秒単位）
    print('dtype = ', data.dtype)            # データ型（量子化ビット数に対応）

    assert data.dtype == 'uint8' or data.dtype == 'int16' or data.dtype == 'int32' or data.dtype == 'float32'
    
    # 最初の10秒分だけ取り出す（元がそれより短ければそのまま）
    nframesNew = min(framerate * 10, nframes)   
    if nchannels == 1:
        dataNew = data[:nframesNew]
    else:
        # 多チャンネル信号なら0番目と1番目の平均値を取り出す
        if data.dtype == 'float32':  # 浮動小数点数のときは [0, 1] の値なので普通に足して2で割る
            dataNew = (data[:nframesNew, 0] + data[:nframesNew, 1])/2
        else: # 整数型のときはオーバーフローする可能性があるので，いったん64bit整数にしてから
            data64 = (data[:nframesNew, 0].astype(np.int64) + data[:nframesNew, 1].astype(np.int64))//2
            dataNew = data64.astype(data.dtype)
    
    return framerate, dataNew

In [ ]:
# WAVE ファイルを入手
#
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar4-ChordC.wav
fn = 'Sound-Guitar4-ChordC.wav'

import os

if not os.path.exists(fn):
    print(f'{fn}のダウンロードがうまくできていないようです')

In [ ]:
# WAVE ファイルを読み込む
#
framerate, dat = readWAVE(fn)
dat = dat.astype(float)
dat -= np.mean(dat)
guitar = dat

---
### 標本化



物理世界を観測して得られる情報は，「時間にともなって変化する気温」のように，時間や空間といった数量にともなって変化する何らかの数量であることが多くあります．
何かの数量の変化にともなって変化する何かの数量において，前者（気温の例における時間を表す量）を「変数」といい，後者（気温）を「値」といいます．

量には，連続なものも離散的な（とびとびの）ものもありますが，コンピュータはディジタルな情報を扱うことしかできませんので，連続な量であっても離散的に表します．
例えば大津市の一地点の気温は，連続な量である時間にともなって変化する連続な量です．しかし，前回扱ったデータは，「1日ごとの平均気温」という値として，時間軸を1日単位で離散化して扱っていました（気温も0.1度刻みの値となっていました）．
音のデータの場合もやはり時間軸が離散化されていましたが，こちらは単位がもっと短い時間でした．

データを扱う際には，このように，本来は連続な変数を離散化してから扱うことがよくあります．変数を離散化するこの処理のことを，**標本化** (sampling) といいます（説明は省きますが，値の方を離散化する処理は **量子化** (quantization) といいます）．
前回の説明では，気温や音のデータはすでに標本化済みで，$\mathbf{x} = (x_0, x_1, \ldots, x_{N-1})$ のように $N$ 個の数値のならんだベクトルとして与えられるとして話をしていました．
しかし，そのようなデータを分析して得られた結果をきちんと解釈するためには，標本化の過程を無視することはできません．

というわけで，標本化について考えてみましょう．

#### 標本化間隔（標本化周期）と標本化周波数

ここでは，変数を一定の間隔で離散化するような標本化の仕方を考えます．この間隔のことを，**標本化間隔** （または **標本化周期**） といいます．

次のセルをそのまま実行すると，$x(t) = \sin{2\pi t}$ （ $t$ の単位は [s] ）の値を $t = 0, 0.1, 0.2, \ldots$ と標本化間隔 $0.1[s]$ で取り出す標本化を行った結果が表示されます． 
灰色の曲線が元の $x(t)$ で，赤い点が標本化によって得られる値です．

標本化間隔は `T` という変数で指定していますので，適当に値を変えて結果を眺めてみましょう．

In [ ]:
T = 0.4

duration = 2.0

t_sample = np.arange(0, duration, T)
N = len(t_sample)
t = np.arange(0, duration, 0.01)
x_sample = np.sin(2*np.pi*t_sample)
x        = np.sin(2*np.pi*t)

print('x = ')
print(x_sample, f'N = {N}')

fig, ax = plt.subplots(figsize=(12, 6))
ax.stem(t_sample, x_sample, linefmt='r--', markerfmt='ro', use_line_collection=True, label='$x_n$')
ax.plot(t, x, '-', label=f'$\sin(2\pi t)$', color='gray')
ax.set_xlabel('t [s]')
ax.axhline(0, color='gray')
ax.set_xlim(-0.05, 2.05)
ax.set_ylim(-1.2, 1.2)
ax.set_xticks(np.arange(0, duration+0.1, T))
ax.legend()

plt.show()

##### やってみよう


標本化間隔を $0.12, 0.2, 0.3, 0.4, 0.5$ などと変えて結果を観察しよう．

標本化間隔の逆数をとったものを，**標本化周波数** といいます．標本化間隔の単位が [s] の場合，標本化周波数の単位は [Hz] です．標本化間隔が 0.1[s] なら，標本化周波数は 1/0.1 = 10 [Hz] です．標本化の際には，標本化間隔を短く（標本化周波数を高く）しないと，元の波形をちゃんと表せません．

#### 標本化間隔・標本化周波数と周波数成分

前回，$\mathbf{x} = (x_0, x_1, \ldots, x_{N-1})$ の離散フーリエ変換

$$
\mathbf{x} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \ldots + c_{N-1}\mathbf{u}_{N-1}
$$

は，$\mathbf{x}$ を次のような成分で展開していることを説明しました．

$$
\begin{array}{ll}
\mbox{定数:} & c_0\mathbf{u}_0\\
\mbox{周期 $N$ の波:} & c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1} \\
\mbox{周期 $N/2$ の波:} & c_2\mathbf{u}_2 + c_{N-2}\mathbf{u}_{N-2} \\
\mbox{周期 $N/3$ の波:} & c_3\mathbf{u}_3 + c_{N-3}\mathbf{u}_{N-3}\\
: & :
\end{array}
$$

これらの成分の中で最も周期が長い（周波数が低い）のは周期 $N$ の波です．対象が時間を変数とするデータで，標本化間隔が $T [s]$ だったとすると，$x_n$ と $x_{n+1}$ の間の時間間隔は $T$ です．したがって，「周期 $N$ の波」 = 「周期 $NT$ [s] の波」，「周期 $\frac{N}{2}$ の波」 = 「周期 $\frac{NT}{2}$ [s] の波」，...，すなわち，「周期 $\frac{N}{k}$ の波」 = 「周期 $\frac{NT}{k}$ [s] の波」（$k=1,2,\ldots$）ということになります．これより，上記を次のように書き直すことができます．

$$
\begin{array}{lll}
k=0 \mbox{の成分:} & \mbox{定数}, & c_0\mathbf{u}_0\\
k=1 \mbox{の成分:} & \mbox{周期 $NT$[s] （周波数 $\frac{1}{NT}$[Hz]）の波} & c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1} \\
k=2 \mbox{の成分:} & \mbox{周期 $\frac{NT}{2}$[s] （周波数 $\frac{2}{NT}$[Hz]）の波} & c_2\mathbf{u}_2 + c_{N-2}\mathbf{u}_{N-2} \\
k=3 \mbox{の成分:} & \mbox{周期 $\frac{NT}{3}$[s] （周波数 $\frac{3}{NT}$[Hz]） の波} & c_3\mathbf{u}_3 + c_{N-3}\mathbf{u}_{N-3}\\
: & :
\end{array}
$$


上の例の離散フーリエ変換でこのことを確認してみましょう．

この例では，$x(t) = \sin{2\pi t}$ という信号を標本化しています．この信号は $t$ が $t = 0$ [s] から $t = 1$ [s] まで進むと1周するので，周期 $1$[s] = 周波数 $1$[Hz] の正弦波です．標本化間隔 $T$ で標本化して得られる $N$ 次元ベクトル $\mathbf{x} = (x_0, x_1, \ldots, x_{n-1})$ に離散フーリエ変換を適用して振幅スペクトルを描いてみると，下図のようになります．

In [ ]:
T = 0.2

duration = 2.0

t_sample = np.arange(0, duration, T)
N = len(t_sample)
t = np.arange(0, duration, 0.01)
x_sample = np.sin(2*np.pi*t_sample)
x        = np.sin(2*np.pi*t)

print('x = ')
print(x_sample, f'N = {N}')

c = fft(x_sample) / N
amp = np.abs(c)
print('|c_k| = ')
print(amp, f'N = {N}')

fig, ax = plt.subplots(1, 2, figsize=(16, 6))


# 横軸 k
ax[0].stem(amp, linefmt='r-', markerfmt='ro', use_line_collection=True, label='$|c_k|$')
ax[0].set_ylim(top=1.0)
ax[0].set_xlabel('k')
ax[0].legend()

# 横軸 周波数[Hz]
f = np.arange(0, N) / (N*T)
ax[1].stem(f, amp, linefmt='r-', markerfmt='ro', use_line_collection=True, label='$|c_k|$')
ax[1].set_ylim(top=1.0)
ax[1].set_xlabel('frequency [Hz]')
ax[1].legend()

plt.show()

print(f'T = {T}, N = {N}')

左右どちらの図も同じ振幅スペクトルを表しますが，左図ではフーリエ係数の番号 $k$ を横軸にとっており，右図では対応する周波数 [Hz] の単位で描いてあります．

例えば，$T = 0.1, N = 20$ の場合，$k=1$ は周波数 $\frac{1}{NT} = 0.5$[Hz] の波の成分，$k=2$ は周波数 $\frac{2}{NT} = 1$[Hz]の波の成分，$k=3$ は周波数 $\frac{3}{NT} = 1.5$[Hz]の波の成分，...，となっています．右図を見ると，元の正弦波の周波数である $1$ [Hz] のところで $|c_k|$ の値が大きくなっています．

このように，離散フーリエ変換で得られた結果を標本化間隔・標本化周波数とあわせて考えると，信号の性質を，元の変数の単位で分析することが可能となります．前回の notebookB や notebookC で登場した横軸の単位が [Hz] の振幅スペクトルも，このようにして描いたものでした．

### 標本化定理


標本化周波数を2倍よりも高くすればok

---
### 高速フーリエ変換

前回説明した通り，離散フーリエ変換とは次のようなものです．

> **離散フーリエ変換**
>
> $N$ 次元ベクトル $\mathbf{u}_0, \mathbf{u}_1, \ldots, \mathbf{u}_{N-1}$ を次のように定めると，$\{\mathbf{u}_0, \mathbf{u}_1, \ldots, \mathbf{u}_{N-1}\}$ は直交基底となる．
>
> $$
\mathbf{u}_k = \left( e^{0\times i\omega_0k}, e^{1\times i\omega_0k}, e^{2\times i\omega_0k}, \ldots, e^{(N-1)\times i\omega_0k} \right) \quad (k = 0, 1, 2, \ldots, N-1) \qquad (9)
$$
>
> ただし，$i = \sqrt{-1}, w_0 = \frac{2\pi}{N}$ である．
> $\{ \mathbf{u}_k \}$ を用いると，任意の $N$ 次元ベクトル $\mathbf{x} = (x_0, x_1, \ldots, x_{N-1})$ を
>
> $$
\mathbf{x} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{N-1}\mathbf{u}_{N-1} \qquad (10)
$$
>
> と一意に展開できる．展開係数 $c_k$ の値は次式で求められる．
>
> $$
c_k = \frac{1}{N}\mathbf{x}\cdot\mathbf{u}_k = \frac{1}{N}\sum_{n=0}^{N-1} x_{n}e^{-i\omega_0kn} \qquad (11)
$$
>
> $\mathbf{x}$ から $\mathbf{c} = (c_0, c_1, \ldots, c_{N-1})$ を求める演算を **離散フーリエ変換** という．また，展開係数のことを **フーリエ係数** という．

コンピュータを用いて離散フーリエ変換の計算を実行する場合，原理的には，式 $(11)$ の計算を実行すればよいだけです．例えば，C言語で離散フーリエ変換を実装すると，次のように書けます（複素数の計算は実部と虚部に分けて行います）．

```
  double x[N];       // 与えられるデータ
  double A[N], B[N]; // フーリエ係数の実部と虚部
  int N = (データ点の数);
  double omega0 = 2*M_PI/(double)N;

  // ここで x に値を代入

  /***** DFT *****/
  for(k = 0; k < N; k++){
    A[k] = B[k] = 0;
    for(n = 0; n < N; n++){
      A[k] += x[n]*cos(omega0*(double)(k*n)); // 実部
      B[k] -= x[n]*sin(omega0*(double)(k*n)); // 虚部
    }
    A[k] /= (double)N; B[k] /= (double)N;
  }
```

見ての通り，三角関数の計算と単純な積や和の計算を繰り返す2重ループです．この構造から想像がつく通り，離散フーリエ変換の計算量は，$N$ の2乗のオーダー（ $O(N^2)$ ）となります．$N$ が $10$倍，$100$倍，...，と大きくなると，計算量は $10^2 = 100$倍，$100^2 = 1$万倍，...，と大きくなっていくということです．

離散フーリエ変換を使う場面では $N$ の大きなデータを扱うことが多いですので，定義通りに計算する上記のようなやり方では計算量が大きすぎます．そのため実用的には，**高速フーリエ変換** (Fast Fourier Transform, FFT) という，離散フーリエ変換の高速演算アルゴリズムが使われます．
高速フーリエ変換のアルゴリズムを用いると，離散フーリエ変換を $O(n\log{n})$ の計算量で実行することができます．

前回も使っていた [SciPy](https://scipy.org/) の関数 [`scipy.fft.fft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html) は，その名前の通り，高速フーリエ変換のアルゴリズムによって離散フーリエ変換を実行するものです．

高速フーリエ変換のアルゴリズムについては，この授業では説明を省略します．高校数学レベルの複素数の知識があれば理解できますので，興味があれば調べてみるとよいです．

高速フーリエ変換のありがたさを体感するために，ちょっと実験してみましょう．

「準備」で読み込んだギター音のデータは，次のようなものです：
- 標本化周波数 8000 [Hz]
- データ数 N = 16384
- 長さ N / 8000 = 2.048 [s]


In [ ]:
N = len(guitar)
print(N)

まずはこのデータの離散フーリエ変換を定義通りに計算した場合の時間を測ってみましょう．

In [ ]:
# 基底を求めておく
U = fft(np.eye(N))

次のセルを実行すると，実行にかかった時間が表示されます．

In [ ]:
%%time
# 離散フーリエ変換を定義通り計算
c = np.conj(U.T) @ guitar
print(c[1])

`CPU times:` の `user` の項が，上記の計算のために CPU が働いた時間です．実行のたびに少し変動しますが，だいたい 1.7 秒くらいかかる結果となるでしょう．
このデータは約2秒の信号ですので，これではリアルタイムの処理（例えば，入力された音をその場でDFT → スペクトルをいじる → IDFTするような処理）は無理ですね．

では，高速フーリエ変換の方は...

In [ ]:
%%timeit -r 10 -n 100
# 高速フーリエ変換（ scipy.fft.fft ）
c = fft(guitar)
#print(c[1])

:速すぎるのでここでは 1000 回実行していますが，1回あたり0.2ミリ秒（= $\frac{0.2}{1000}$ = $\frac{1}{5000}$秒）以下で済んでいます．